# This NB demonstrates how we can use methods from visualizations

We can use the method to plot all patterns, double click on specific patterns to highlight them in the heatmaps, and select top patterns to plot them on their own.

In [1]:
# python imports
import pandas as pd
import altair as alt

# imports to load relationships
import requests
from pandas.io.json import json_normalize

# our imports
%load_ext autoreload
%autoreload 2
%cd ../intervals/
from main import *
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


In [2]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
prefix = "CRIM_Model_00"
files = ["17"] 
postfix = ".mei"

## `plot_ngrams_heatmap`

First, we do the normal steps to get a ngrams dataframe.

In [3]:
corpus = CorpusBase([root + prefix + files[0] + postfix])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5, cell_type=str)
mel_ngrams

Requesting file from https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0017.mei...
Successfully imported.


,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,NaN,"1, 2, 2, 2, -4",NaN,NaN,NaN,NaN
7.0,NaN,"2, 2, 2, -4, 1",NaN,NaN,NaN,NaN
8.0,"1, 2, 1, 2, -3","2, 2, -4, 1, -2",NaN,NaN,NaN,NaN
10.0,NaN,"2, -4, 1, -2, -2",NaN,NaN,NaN,NaN
12.0,NaN,"-4, 1, -2, -2, 2",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1466.0,NaN,NaN,"2, -5, 2, 2, 2",NaN,NaN,NaN
1468.0,NaN,NaN,"-5, 2, 2, 2, 2",NaN,NaN,NaN
1469.0,NaN,NaN,"2, 2, 2, 2, -5",NaN,NaN,NaN
1470.0,NaN,NaN,"2, 2, 2, -5, 5",NaN,NaN,NaN


Can read the documentation with `help(viz.plot_ngrams_heatmap)`

In [4]:
help(viz.plot_ngrams_heatmap)

Help on function plot_ngrams_heatmap in module visualizations:

plot_ngrams_heatmap(ngrams_df, model=None, selected_pattern=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot a heatmap for crim-intervals getNgram's output.
    :param ngrams_df: crim-intervals getNgram's output
    :param model: if not None, rely on the model to calculate the durations of patterns
    of just outputing only offsets (default=False).
    :param selected_pattern: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: the height of the final heatmap (optional)
    :return: a bar chart that displays the different patterns and their counts,
    and a heatmap with the start offsets of chosen voices / patterns



In [5]:
viz.plot_ngrams_heatmap(mel_ngrams,selected_pattern=[], voices=[])

alt.VConcatChart(...)

### Selecting patterns

We could only include some patterns in the heatmaps

First, I collected the top patterns and turned these patterns into a list.

In [6]:
# count and get the 10 most popular patterns
mel_ngrams_top_patterns_df = mel_ngrams.stack().value_counts().to_frame().head(10)
# retrieve a list to generate heatmaps
mel_ngrams_top_patterns_list = mel_ngrams_top_patterns_df.index.to_list()
mel_ngrams_top_patterns_df

,0
"-2, -2, -2, -2, -2",21
"2, 2, 2, 2, 2",20
"2, 2, 2, 2, -2",16
"2, 2, 2, 2, -3",14
"2, 2, 2, -3, 2",13
"2, 2, -2, -3, 2",13
"-2, 2, 2, 2, 2",11
"2, -2, -3, 2, -2",11
"-2, -3, 2, 2, 2",10
"1, 2, 2, -2, -3",10


In [7]:
mel_ngrams_top_patterns_list 

['-2, -2, -2, -2, -2',
 '2, 2, 2, 2, 2',
 '2, 2, 2, 2, -2',
 '2, 2, 2, 2, -3',
 '2, 2, 2, -3, 2',
 '2, 2, -2, -3, 2',
 '-2, 2, 2, 2, 2',
 '2, -2, -3, 2, -2',
 '-2, -3, 2, 2, 2',
 '1, 2, 2, -2, -3']

We would pass the list of patterns into the `patterns` parametter.

In [8]:
viz.plot_ngrams_heatmap(mel_ngrams, selected_pattern=mel_ngrams_top_patterns_list)

alt.VConcatChart(...)

**Cross-checking output from the heatmap with the ngrams dataframe**

We can see that'1, 1, 1, 1, 1' is a really popular pattern. Let's view its start and end points in ngram and in the heatmap's dataframe to make sure that it has been calculated correctly

### Heatmaps with for ngrams!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

#### First, we correctly calculate the durations using `getDuration` from crim-intervals

In [9]:
dur = model.getDuration(mel)
mel.reindex_like(dur)
dur_ngrams = model.getNgrams(df=dur, n=5)

In [10]:
dur_ngrams = dur_ngrams.applymap(lambda cell: sum(cell) if pd.notna(cell) else cell)
dur_ngrams 

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,NaN,NaN,16.0,19.0,17.0,18.0
4.0,NaN,12.0,NaN,NaN,NaN,NaN
7.0,NaN,10.0,NaN,NaN,NaN,NaN
8.0,40.0,34.0,48.0,40.0,38.0,40.0
10.0,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1466.0,NaN,NaN,6.0,NaN,NaN,NaN
1468.0,NaN,NaN,6.0,20.0,NaN,NaN
1469.0,NaN,NaN,21.0,NaN,NaN,NaN
1470.0,NaN,NaN,40.0,NaN,NaN,NaN


In [11]:
viz.plot_ngrams_heatmap(mel_ngrams, dur_ngrams , selected_pattern=mel_ngrams_top_patterns_list, voices=[])

alt.VConcatChart(...)

## `plot_close_match_heatmap`

In [12]:
help(viz.plot_close_match_heatmap)

Help on function plot_close_match_heatmap in module visualizations:

plot_close_match_heatmap(ngrams_df, key_pattern, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot how closely the other vectors match a selected vector.
    Uses the Levenshtein distance.
    :param ngrams_df: crim-intervals getNgram's output
    :param key_pattern: a pattern the users selected to compare other patterns with (tuple of floats)
    :param selected_pattern: the specific other vectors the users selected
    :param ngrams_duration: if None, simply output the offsets. If the users input a
    list of durations, caculate the end by adding durations with offsets and
    display the end on the heatmap accordingly.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: t

In [13]:
viz.plot_close_match_heatmap(mel_ngrams, '-2, -2, -2, -2, -2', selected_patterns=[], voices=[])

alt.Chart(...)

In [14]:
# use the durations calculated from above
viz.plot_close_match_heatmap(mel_ngrams,'-2, -2, -2, -2, -2', dur_ngrams, selected_patterns=[], voices=[])

alt.Chart(...)

## `plot_relationship_heatmap`

In [15]:
data_relationships = requests.get('http://crimproject.org/data/relationships/').json()
#df = pd.DataFrame(data)
df_relationships = pd.json_normalize(data_relationships)
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_test = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[1]].copy()

In [16]:
help(viz.plot_relationship_heatmap)

Help on function plot_relationship_heatmap in module visualizations:

plot_relationship_heatmap(df, ema_col, category1='musical_type', category0='observer.name', option=1, heat_map_width=800, heat_map_height=300)
    This method plots a chart relationships/observations dataframe retrieved from their
    corresponding json files. This chart has two bar charts displaying the count of variables
    the users selected, and a heatmap displaying the locations of the relationship.
    :param df: relationships or observations dataframe
    :param ema_col: name of the ema column
    :param category1: name of the first category for the first bar chart.
    (default='musical_type')
    :param category0: name of the zeroth category for the zeroth bar chart.
    (default='observer.name')
    :param option: if 0, the charts would be colored based on category0,
    if 1, all of the charts would be colored based on category1. (default=1)
    :param heat_map_width: the width of the final heatmap (defau

In [17]:
viz.plot_relationship_heatmap(df_relationships_test, 'model_observation.ema', category1='musical_type', category0='observer.name', option=1, heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)